In [99]:
filename = "./dataset/forCLFnohash1000.csv"
with open(filename) as f:
    content = f.readlines()

##Remove Labels
del(content[0])

##Remove '\n' elements
content = [x for x in content if x != '\n']

##Striping Whitespaces
content = [x.strip() for x in content]

##Spilting by ','
content = [x.split(',') for x in content]
##print(content)

In [100]:
##docs = [x[0] for x in content]
docs = [x[0] for x in content[0:125]] + [x[0] for x in content[125:250]]
##print(docs)

In [101]:
from pythainlp.corpus import stopwords
from pythainlp.tokenize import word_tokenize
import re

def tokenizer(sentence):
    pattern = re.compile(r"[^\u0E00-\u0E4C]|^'|'$|''|\u0E46|\u0E3F")
    char_to_remove = re.findall(pattern, sentence)
    list_with_char_removed = [char for char in sentence if not char in char_to_remove]
    result_string = ''.join(list_with_char_removed)
    return word_tokenize(result_string, engine='deepcut')

In [102]:
tokenized_docs = [tokenizer(x) for x in docs]
##print(tokenized_docs)

In [103]:
word_set = [set(x) for x in tokenized_docs]
##print(word_set)

In [104]:
w = word_set[0].union(word_set[1])
for x in range(len(word_set)):
    w = w.union(word_set[x])
##'โมโห' in w

In [105]:
wordDictList = []
for doc in tokenized_docs:
    wordDict = dict.fromkeys(w,0)
    for word in doc:
        wordDict[word]+=1
    wordDictList.append(wordDict)
##wordDictList
##'โมโห' in wordDictList[0]

In [106]:
import pandas as pd
pd.DataFrame(wordDictList)

,,กกกกก,กกกกกกจิก,กฎหมาย,กทม,กรอบ,กระดาษ,กระตุกก,กระทง,กระทืบ,...,ไส้,ไห,ไหน,ไหม,ไหมม,ไหร่,ไหว,ไอ,ไอพี,ไอ่เด็ก
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [108]:
tfDict_list = []
for word,doc in list(zip(wordDictList, tokenized_docs)):
    tfDict_list.append(computeTF(word,doc))
##tfDict_list

In [109]:
def computeIdf(docList):
    import math
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word]+=1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

In [110]:
idfs = computeIdf(wordDictList)

In [111]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word,val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [112]:
tfidf_list = []
for tfDict in tfDict_list:
    tfidf_list.append(computeTFIDF(tfDict,idfs))
##tfidf_list

In [113]:
import pandas as pd
pd.DataFrame(tfidf_list)

,,กกกกก,กกกกกกจิก,กฎหมาย,กทม,กรอบ,กระดาษ,กระตุกก,กระทง,กระทืบ,...,ไส้,ไห,ไหน,ไหม,ไหมม,ไหร่,ไหว,ไอ,ไอพี,ไอ่เด็ก
0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.22992,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,0.0,0.0,0.78878,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
6,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
7,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [114]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

X = []
for l in tfidf_list:
    temp = []
    for key, val in l.items():
        temp.append(val)
    X.append(temp)

X = np.array(X)
##X = normalize(X)

scaler = MinMaxScaler()
X = scaler.fit(X).transform(X)

y = [ 0 for i in range(len(tokenized_docs))]
for i in range(int(len(tokenized_docs)/2)):
    y[i]=1
y = np.array(y)

##print(X)

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)
##skf.get_n_splits(X, y)
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tclf = svm.LinearSVC().fit(X_train,y_train)
    predict = tclf.predict(X_test)
    
    print(predict)
    print(y_test)
    print(recall_score(y_test,predict),precision_score(y_test,predict),f1_score(y_test,predict))

[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.64 0.6956521739130435 0.6666666666666666
[1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.52 0.7222222222222222 0.6046511627906976
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.72 0.75 0.7346938775510204
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8 0.6451612903225806 0.7142857142857142
[1 0 1 1 1 0 1 0 1 

In [115]:
## Create Emotion Dictionary
angry_dict = [
    'โทสะ','วิโรธ','ความโกรธ','ความฉุนเฉียว','ความขัดเคือง','ขัดแค้น','ขึ้งเคียด','ขึ้งโกรธ',
    'ดาลเดือด','ทำให้โกรธ','ทำให้ฉุนเฉียว','ทำให้เกิดโทสะ','ยั่วโมโห','โกรธ','ฉุนเฉียว','เคือง',
    'แค้น','เดือด','โมโห']
anticipate_dict = [
    'คาด','ตั้งตาคอย','หยั่งทราบ','คาดหมาย','มุ่งหวัง','ชิงทำ','ทำล่วงหน้า','คาดคิด','คาดหวัง',
    'คาดเดา','ตั้งตารอ','ประมาณการณ์','มาดหมาย','เป็นที่คาดหวัง']

In [151]:
def findDictFeature(dictionary):
    N = len(wordDictList)
    dict_feature = [0 for i in range(N)]
    for idx, wordDict in enumerate(wordDictList):
        for word, val in wordDict.items():
            if val > 0:
                if word in dictionary:
                    dict_feature[idx] = 1
                    print(idx,word)
    return np.array(dict_feature).reshape(N,1)

In [117]:
angry_dict_feature = findDictFeature(angry_dict)

25 โมโห
68 โกรธ
88 โกรธ
150 โกรธ


In [118]:
anticipate_dict_feature = findDictFeature(anticipate_dict)

161 คาด
163 คาดหวัง


In [119]:
Xnew = np.concatenate((X,angry_dict_feature,anticipate_dict_feature), axis=1)

In [120]:
##Xnew = normalize(Xnew)
##scaler = MinMaxScaler()
##Xnew = scaler.fit(Xnew).transform(Xnew)

adf = normalize(angry_dict_feature)
scaler = MinMaxScaler()
adf = scaler.fit(adf).transform(adf)

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)
##skf.get_n_splits(X, y)
for train_index, test_index in skf.split(adf, y):
    #print("TRAIN:", X[train_index], "TEST:", X[test_index])
    X_train, X_test = adf[train_index], adf[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tclf = svm.LinearSVC().fit(X_train,y_train)
    predict = tclf.predict(X_test)
    
    print(predict)
    print(y_test)
    print(recall_score(y_test,predict),precision_score(y_test,predict),f1_score(y_test,predict))

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.04 1.0 0.07692307692307693
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.0 0.0 0.0
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.04 0.5 0.07407407407407407
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.04 1.0 0.07692307692307693
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0

/Users/nopchamnan.n.aa/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nopchamnan.n.aa/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [121]:
angry_dict_feature.shape

(250, 1)

In [122]:
test = np.zeros(250).reshape(250,1)
for i in range (250):
    if i<125: 
        test[i]=1
    else: 
        test[i]=0
test.shape

(250, 1)

In [123]:
Xnew2 = np.concatenate((Xnew,test), axis=1)

In [124]:
scaler = MinMaxScaler()
Xnew = scaler.fit(Xnew).transform(Xnew)
skf2 = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)

for train_index, test_index in skf.split(Xnew, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train2, X_test2 = Xnew[train_index], Xnew[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    tclf = svm.LinearSVC().fit(X_train2,y_train2)
    predict2 = tclf.predict(X_test2)
    
    print(predict2)
    print(y_test2)
    print(recall_score(y_test2,predict2),precision_score(y_test2,predict2),f1_score(y_test2,predict2))

[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.64 0.6956521739130435 0.6666666666666666
[1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.52 0.7222222222222222 0.6046511627906976
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.72 0.75 0.7346938775510204
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.84 0.65625 0.736842105263158
[1 0 1 1 1 0 1 0 1 0 1 0 1 1 1

In [133]:
def train_model(X,y,n):
    R = 0.0
    P = 0.0
    F = 0.0
    
    skf = StratifiedKFold(n_splits=n, random_state=30, shuffle=True)
    
    for train_index, test_index in skf.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        tclf = svm.LinearSVC().fit(X_train,y_train)
        predict = tclf.predict(X_test)
        
        r=recall_score(y_test,predict)
        p=precision_score(y_test,predict)
        f=f1_score(y_test,predict)
        
        R = R + r
        P = P + p
        F = F + f
        
        print('Prediction')
        print(predict)
        print('y')
        print(y_test)
        print('scores: recall, precision, f1')
        print(r,p,f)
        print()
        
    print()
    print('Average')
    print(R/n,P/n,F/n)

In [135]:
train_model(X,y,5)

Prediction
[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.64 0.6956521739130435 0.6666666666666666

Prediction
[1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.52 0.7222222222222222 0.6046511627906976

Prediction
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.72 0.75 0.7346938775510204

Prediction
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 

In [136]:
train_model(Xnew,y,5)

Prediction
[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.64 0.6956521739130435 0.6666666666666666

Prediction
[1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.52 0.7222222222222222 0.6046511627906976

Prediction
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.72 0.75 0.7346938775510204

Prediction
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 

In [141]:
## Create dictionary from NRC Emotion Lexicon
fname = './dataset/NRC-Emotion-Lexicon.xlsx'
df = pd.read_excel(fname)

emowords = np.array(df['Thai'])
emowords

array(['ผงะ', 'ลูกคิด', 'ละทิ้ง', ..., 'เกี่ยวกับสัตววิทยา', 'สัตววิทยา',
       'ซูม'], dtype=object)

In [143]:
emowords.shape

(14182,)

In [152]:
emodict = findDictFeature(emowords)

0 คน
0 มือ
0 ตีน
0 กระทืบ
0 รวม
0 คัน
0 ปาก
2 ติด
3 วัน
3 บ้าน
3 แตก
4 ช่อง
4 มี
4 ห้อง
5 ดี
5 สลัด
5 มา
6 คลื่น
6 ดี
6 ที่สุด
6 เสีย
6 ระบบ
6 อยู่
6 ประมูล
6 ให้
6 พัฒนา
6 เงิน
6 มา
6 ตัง
7 รู้จัก
7 คน
7 เห็น
7 ใช้
7 ให้
7 แท้จริง
7 ใช่
7 อย่าง
7 ตลก
8 ให้
8 พยาบาล
8 ส่วน
8 ฟัง
9 ขัดใจ
10 ลืม
10 เพื่อน
11 แรก
11 เขา
11 เรื่อง
11 สัญญา
11 บอก
11 ตอน
11 กับ
11 อย่าง
12 นม
12 มี
12 อยู่
13 หัว
13 ชีวิต
13 ทำไม
14 ตี
15 เกลียด
15 เตือน
15 ส่ง
15 มี
15 เกม
15 เห็น
15 แจ้ง
15 ดีใจ
15 ไอ
15 ตลอด
15 ทำ
15 เล่น
15 ให้
15 บ้า
15 มา
16 คน
16 ตอบ
16 กีฬา
16 เลือก
16 ให้
16 เป็น
16 ภาษา
16 ประธาน
17 เดิน
17 แรก
17 ปิด
17 ตรง
17 ออก
17 มี
17 ให้
17 ประตู
17 ตอน
17 ผ้า
17 ต่อ
17 มา
17 หน้า
17 ทาง
17 ห้อง
18 รู้สึก
18 ตาม
18 ทำ
18 ให้
18 เลิก
19 เป็น
20 ตอบ
21 วัน
21 ดู
22 หวัง
24 เลิก
24 ชอบ
25 วัน
25 สาย
25 แบบ
25 บ้าน
25 ช่าง
25 สั่ง
25 ทำ
25 มาก
25 ให้
25 บอก
25 ดัน
25 มา
25 รั้ว
25 พอ
25 งาน
26 กระดาษ
26 ละเลย
26 ทำไม
26 กับ
26 เศษ
26 มา
26 ใบ
26 ค่า
26 เรียน
27 คำนวณ
27 มี
27 อัตนัย
28 ทอด
28 ม

In [153]:
emodict

array([[1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [154]:
emodict.shape

(250, 1)

In [155]:
train_model(emodict,y,5)

Prediction
[1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.92 0.5 0.647887323943662

Prediction
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
1.0 0.5 0.6666666666666666

Prediction
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.96 0.4897959183673469 0.6486486486486486

Prediction
[1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 1 1 0 1 1 1 1 1 1 1 1 1]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [180]:
def findDictFeatureNormalized(dictionary):
    N = len(wordDictList)
    count = 0.0
    dict_feature = [0 for i in range(N)]
    for idx, wordDict in enumerate(wordDictList):
        for word, val in wordDict.items():
            if val > 0:
                if word in dictionary:
                    dict_feature[idx] = dict_feature[idx] + 1
                    print(idx,word,val)
    for idx, doc in enumerate(tokenized_docs):
        n = len(doc)
        dict_feature[idx] = dict_feature[idx]/n
    return np.array(dict_feature).reshape(N,1)

In [181]:
emodictnormalized = findDictFeatureNormalized(emowords)

0 คน 1
0 มือ 1
0 ตีน 1
0 กระทืบ 1
0 รวม 1
0 คัน 4
0 ปาก 1
2 ติด 1
3 วัน 1
3 บ้าน 1
3 แตก 1
4 ช่อง 1
4 มี 1
4 ห้อง 1
5 ดี 1
5 สลัด 1
5 มา 1
6 คลื่น 1
6 ดี 1
6 ที่สุด 1
6 เสีย 1
6 ระบบ 1
6 อยู่ 1
6 ประมูล 2
6 ให้ 1
6 พัฒนา 1
6 เงิน 1
6 มา 3
6 ตัง 1
7 รู้จัก 1
7 คน 1
7 เห็น 1
7 ใช้ 1
7 ให้ 1
7 แท้จริง 1
7 ใช่ 1
7 อย่าง 1
7 ตลก 1
8 ให้ 1
8 พยาบาล 1
8 ส่วน 1
8 ฟัง 1
9 ขัดใจ 1
10 ลืม 1
10 เพื่อน 1
11 แรก 1
11 เขา 1
11 เรื่อง 1
11 สัญญา 1
11 บอก 1
11 ตอน 1
11 กับ 1
11 อย่าง 1
12 นม 1
12 มี 1
12 อยู่ 1
13 หัว 1
13 ชีวิต 1
13 ทำไม 1
14 ตี 1
15 เกลียด 1
15 เตือน 1
15 ส่ง 1
15 มี 1
15 เกม 2
15 เห็น 1
15 แจ้ง 1
15 ดีใจ 1
15 ไอ 1
15 ตลอด 1
15 ทำ 1
15 เล่น 1
15 ให้ 1
15 บ้า 1
15 มา 2
16 คน 1
16 ตอบ 1
16 กีฬา 1
16 เลือก 1
16 ให้ 1
16 เป็น 3
16 ภาษา 1
16 ประธาน 1
17 เดิน 1
17 แรก 1
17 ปิด 2
17 ตรง 1
17 ออก 1
17 มี 1
17 ให้ 1
17 ประตู 1
17 ตอน 1
17 ผ้า 1
17 ต่อ 1
17 มา 1
17 หน้า 1
17 ทาง 1
17 ห้อง 2
18 รู้สึก 1
18 ตาม 1
18 ทำ 1
18 ให้ 1
18 เลิก 1
19 เป็น 1
20 ตอบ 1
21 วัน 1
21 ดู 1
22 หวัง 1
24 เลิก 1


In [182]:
emodictnormalized

array([[0.33333333],
       [0.        ],
       [0.2       ],
       [0.33333333],
       [0.5       ],
       [0.42857143],
       [0.33333333],
       [0.39130435],
       [0.44444444],
       [0.2       ],
       [0.66666667],
       [0.28571429],
       [0.27272727],
       [0.25      ],
       [0.33333333],
       [0.39473684],
       [0.30769231],
       [0.375     ],
       [0.27777778],
       [0.125     ],
       [0.11111111],
       [0.2       ],
       [0.11111111],
       [0.        ],
       [0.33333333],
       [0.34883721],
       [0.34615385],
       [0.75      ],
       [0.23076923],
       [0.28571429],
       [1.        ],
       [0.28125   ],
       [0.25      ],
       [0.        ],
       [0.3       ],
       [0.22222222],
       [0.33333333],
       [0.38461538],
       [0.33333333],
       [0.44444444],
       [0.3       ],
       [0.36585366],
       [0.48275862],
       [0.125     ],
       [0.25      ],
       [0.25      ],
       [0.1       ],
       [0.25 

In [183]:
train_model(emodictnormalized,y,5)

Prediction
[0 1 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 1
 1 0 0 1 0 0 1 1 1 1 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.44 0.4583333333333333 0.4489795918367347

Prediction
[0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 1 1 0
 0 1 1 1 0 0 0 0 0 1 0 1 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.48 0.5217391304347826 0.4999999999999999

Prediction
[0 0 1 1 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0
 1 1 0 1 1 1 1 1 0 1 1 1 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.48 0.42857142857142855 0.4528301886792452

Prediction
[1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1
 0 0 0 1 1 0 0 0 0 0 1 1 0]
y
[1 1

In [184]:
## concatenate emodict with tfidf

Xemo = np.concatenate((X,emodict), axis=1)
train_model(Xemo,y,5)

Prediction
[0 1 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 0 1 0
 0 0 1 1 1 0 0 0 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.64 0.6666666666666666 0.6530612244897959

Prediction
[1 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.6 0.7894736842105263 0.6818181818181819

Prediction
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.72 0.72 0.72

Prediction
[1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 0
 1 1 1 0 0 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [185]:
## concatenate emodictnormalized with tfidf

XemoN = np.concatenate((X,emodictnormalized), axis=1)
train_model(XemoN,y,5)

Prediction
[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.64 0.7619047619047619 0.6956521739130435

Prediction
[1 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.6 0.7894736842105263 0.6818181818181819

Prediction
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.72 0.72 0.72

Prediction
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [189]:
angrywords = np.array([df['Thai'],df['Anger']])
angrywords

array([['ผงะ', 'ลูกคิด', 'ละทิ้ง', ..., 'เกี่ยวกับสัตววิทยา',
        'สัตววิทยา', 'ซูม'],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [190]:
angrywords.shape

(2, 14182)

In [193]:
angrywords[1][0]

0

In [201]:
angrywordslist = []
for i in range(len(angrywords[1])):
    if angrywords[1][i] == 1:
        angrywordslist.append(angrywords[0][i])
angrywordslist = np.array(angrywordslist).reshape(len(angrywordslist),1)

In [202]:
angrywordslist.shape

(1247, 1)

In [204]:
angry = findDictFeatureNormalized(angrywordslist)

0 กระทืบ 1
3 แตก 1
6 เงิน 1
9 ขัดใจ 1
11 สัญญา 1
11 กับ 1
14 ตี 1
15 เกลียด 1
15 บ้า 1
18 เลิก 1
24 เลิก 1
25 รั้ว 2
26 กับ 1
29 กับ 1
31 ทิ้ง 1
31 บ่น 2
32 ร้อน 1
34 ป่วย 1
36 เตะ 1
37 ระเบิด 1
40 ดัง 1
40 ขโมย 1
40 ดูถูก 1
42 กับ 2
46 ยิง 1
50 บ้า 1
54 จับ 1
54 กับ 1
56 กับ 1
58 กับ 1
60 ขโมย 1
64 ข้าม 1
68 โกรธ 1
69 บ่น 1
71 ร้าย 1
72 กับ 1
74 บุก 1
85 ยุง 1
85 ตบ 1
85 แรง 1
88 โกรธ 1
92 กระทืบ 1
94 ตบ 1
96 เงิน 1
99 เงิน 1
101 ทำร้าย 1
105 ยิง 1
108 ป่วย 1
108 บ่น 2
113 ตบ 1
116 รุนแรง 1
116 โรค 1
123 บ่น 1
124 กับ 2
127 บ้า 1
129 แถว 1
131 อันดับ 1
139 กับ 1
150 โกรธ 1
150 คำ 1
154 เงิน 1
156 กับ 1
161 กับ 1
168 ตื่นเต้น 1
179 กับ 1
181 ท้าทาย 1
196 เลิก 1
199 สู้ 1
215 กับ 1
215 ใต้ 1
222 กับ 1
226 กับ 1
244 กับ 1
246 เผื่อ 1
247 ใต้ 1
248 กับ 1


In [206]:
train_model(angry,y,5)

Prediction
[1 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 1 1 0 0 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.32 0.6666666666666666 0.43243243243243246

Prediction
[0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.36 0.6923076923076923 0.47368421052631576

Prediction
[0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 1]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.36 0.5625 0.43902439024390244

Prediction
[1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1]
y
[1 1 1 1 1 1 1

In [207]:
Xangry = np.concatenate((X,angry),axis=1)
train_model(Xangry,y,5)

Prediction
[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.6 0.6818181818181818 0.6382978723404256

Prediction
[1 0 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.56 0.7368421052631579 0.6363636363636364

Prediction
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
scores: recall, precision, f1
0.72 0.72 0.72

Prediction
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
y
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1